# OpenAIEmbeddings

문서 임베딩은 문서의 내용을 수치적인 벡터로 변환하는 과정입니다. 

이 과정을 통해 문서의 의미를 수치화하고, 다양한 자연어 처리 작업에 활용할 수 있습니다. 대표적인 사전 학습된 언어 모델로는 BERT와 GPT가 있으며, 이러한 모델들은 문맥적 정보를 포착하여 문서의 의미를 인코딩합니다. 

문서 임베딩은 토큰화된 문서를 모델에 입력하여 임베딩 벡터를 생성하고, 이를 평균하여 전체 문서의 벡터를 생성합니다. 이 벡터는 문서 분류, 감성 분석, 문서 간 유사도 계산 등에 활용될 수 있습니다.

[더 알아보기](https://platform.openai.com/docs/guides/embeddings/embedding-models)

## 설정

먼저 langchain-openai를 설치하고 필요한 환경 변수를 설정합니다.


지원되는 모델 목록

| MODEL                  | PAGES PER DOLLAR | PERFORMANCE ON MTEB EVAL | MAX INPUT | Embedding size |
|------------------------|------------------|---------------------------|-----------|------------|
| text-embedding-3-small | 62,500           | 62.3%                     | 8191      | 1536       |
| text-embedding-3-large | 9,615            | 64.6%                     | 8191      | 3072       |
| text-embedding-ada-002 | 12,500           | 61.0%                     | 8191      | 1536       |



In [1]:
#실습용 AOAI 환경변수 읽기
import os

AOAI_ENDPOINT=os.getenv("AOAI_ENDPOINT")
AOAI_API_KEY=os.getenv("AOAI_API_KEY")
AOAI_DEPLOY_GPT4O=os.getenv("AOAI_DEPLOY_GPT4O")
AOAI_DEPLOY_GPT4O_MINI=os.getenv("AOAI_DEPLOY_GPT4O_MINI")
AOAI_DEPLOY_EMBED_3_LARGE=os.getenv("AOAI_DEPLOY_EMBED_3_LARGE")
AOAI_DEPLOY_EMBED_3_SMALL=os.getenv("AOAI_DEPLOY_EMBED_3_SMALL")
AOAI_DEPLOY_EMBED_ADA=os.getenv("AOAI_DEPLOY_EMBED_ADA")

In [2]:
from langchain_openai import AzureOpenAIEmbeddings

# OpenAI 임베딩을 생성합니다.
embeddings = AzureOpenAIEmbeddings(model=AOAI_DEPLOY_EMBED_3_LARGE,
    openai_api_version="2024-02-01",
    api_key= AOAI_API_KEY,  
    azure_endpoint=AOAI_ENDPOINT
    )

In [3]:
text = "임베딩 테스트를 하기 위한 샘플 문장입니다."

## 쿼리 임베딩


`embeddings.embed_query(text)`는 주어진 텍스트를 임베딩 벡터로 변환하는 함수입니다.

이 함수는 텍스트를 벡터 공간에 매핑하여 의미적으로 유사한 텍스트를 찾거나 텍스트 간의 유사도를 계산하는 데 사용될 수 있습니다.

In [4]:
# 텍스트를 임베딩하여 쿼리 결과를 생성합니다.
query_result = embeddings.embed_query(text)

In [5]:
len(query_result)

3072

`query_result[:5]`는 `query_result` 리스트의 처음 5개 요소를 슬라이싱(slicing)하여 선택합니다.

In [6]:
# 쿼리 결과의 처음 5개 항목을 선택합니다.
query_result[:5]

[-0.00311934482306242,
 -0.007872357964515686,
 -0.012800268828868866,
 0.030090242624282837,
 0.016375118866562843]

## Document 임베딩


`embeddings.embed_documents()` 함수를 사용하여 텍스트 문서를 임베딩합니다.

- `[text]`를 인자로 전달하여 단일 문서를 리스트 형태로 임베딩 함수에 전달합니다.
- 함수 호출 결과로 반환된 임베딩 벡터를 `doc_result` 변수에 할당합니다.


In [6]:
doc_result = embeddings.embed_documents(
    [text, text, text, text]
)  # 텍스트를 임베딩하여 문서 벡터를 생성합니다.

`doc_result[0][:5]`는 `doc_result` 리스트의 첫 번째 요소에서 처음 5개의 문자를 슬라이싱하여 선택합니다.


In [7]:
len(doc_result)  # 문서 벡터의 길이를 확인합니다.

4

In [9]:
# 문서 결과의 첫 번째 요소에서 처음 5개 항목을 선택합니다.
doc_result[0][:5]

[-0.00311934482306242,
 -0.007872357964515686,
 -0.012800268828868866,
 0.030090242624282837,
 0.016375118866562843]

## 차원 지정

`text-embedding-3` 모델 클래스를 사용하면 반환되는 임베딩의 크기를 지정할 수 있습니다.

예를 들어, 기본적으로 `text-embedding-3-small`는 1536 차원의 임베딩을 반환합니다.


In [8]:
# 문서 결과의 첫 번째 요소의 길이를 반환합니다.
len(doc_result[0])

3072

### 차원(dimensions) 조정

하지만 `dimensions=1024`를 전달함으로써 임베딩의 크기를 1024로 줄일 수 있습니다.

In [9]:
# OpenAI의 "text-embedding-3-small" 모델을 사용하여 1024차원의 임베딩을 생성하는 객체를 초기화합니다.
embeddings_1024 = AzureOpenAIEmbeddings(model=AOAI_DEPLOY_EMBED_3_LARGE,
    openai_api_version="2024-02-01",
    api_key= AOAI_API_KEY,  
    azure_endpoint=AOAI_ENDPOINT,
    dimensions=1024)

In [10]:
# 주어진 텍스트를 임베딩하고 첫 번째 임베딩 벡터의 길이를 반환합니다.
len(embeddings_1024.embed_documents([text])[0])

1024

## 유사도 계산


In [13]:
!pip install scikit-learn

In [11]:
sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 만나서 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like to eat apples."

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]
embedded_sentences = embeddings_1024.embed_documents(sentences)

In [13]:
def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

In [14]:
# sentence1 = "안녕하세요? 반갑습니다."
# sentence2 = "안녕하세요? 만나서 반가워요."
# sentence3 = "Hi, nice to meet you."
# sentence4 = "I like to eat apples."

for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

[유사도 0.9657] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 반갑습니다!
[유사도 0.8571] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.5445] 안녕하세요? 반갑습니다. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1725] 안녕하세요? 반갑습니다. 	 <=====> 	 I like to eat apples.
[유사도 0.8406] 안녕하세요? 반갑습니다! 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.5340] 안녕하세요? 반갑습니다! 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1875] 안녕하세요? 반갑습니다! 	 <=====> 	 I like to eat apples.
[유사도 0.5869] 안녕하세요? 만나서 반가워요. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1596] 안녕하세요? 만나서 반가워요. 	 <=====> 	 I like to eat apples.
[유사도 0.2485] Hi, nice to meet you. 	 <=====> 	 I like to eat apples.


# Chroma

이 노트북에서는 Chroma 벡터스토어를 시작하는 방법을 다룹니다.

Chroma는 개발자의 생산성과 행복에 초점을 맞춘 AI 네이티브 오픈 소스 벡터 데이터베이스입니다. Chroma는 Apache 2.0에 따라 라이선스가 부여됩니다. 


**참고링크**

- [Chroma LangChain 문서](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)
- [Chroma 공식문서](https://docs.trychroma.com/getting-started)
- [LangChain 지원 VectorStore 리스트](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)

샘플 데이터셋을 로드합니다.

In [ ]:
#실습용 AOAI 환경변수 읽기
import os

AOAI_ENDPOINT=os.getenv("AOAI_ENDPOINT")
AOAI_API_KEY=os.getenv("AOAI_API_KEY")
AOAI_DEPLOY_GPT4O=os.getenv("AOAI_DEPLOY_GPT4O")
AOAI_DEPLOY_GPT4O_MINI=os.getenv("AOAI_DEPLOY_GPT4O_MINI")
AOAI_DEPLOY_EMBED_3_LARGE=os.getenv("AOAI_DEPLOY_EMBED_3_LARGE")
AOAI_DEPLOY_EMBED_3_SMALL=os.getenv("AOAI_DEPLOY_EMBED_3_SMALL")
AOAI_DEPLOY_EMBED_ADA=os.getenv("AOAI_DEPLOY_EMBED_ADA")

In [ ]:
#!pip install langchain-chroma

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma


# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("./data/vector_store/nlp-keywords.txt")
loader2 = TextLoader("./data/vector_store/finance-keywords.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1), len(split_doc2)

(11, 6)

In [ ]:
# OpenAI 임베딩을 생성합니다.
embeddings = AzureOpenAIEmbeddings(model=AOAI_DEPLOY_EMBED_3_LARGE,
    openai_api_version="2024-02-01",
    api_key= AOAI_API_KEY,  
    azure_endpoint=AOAI_ENDPOINT
    )

## VectorStore 생성

### 벡터 저장소 생성 (from_documents)

`from_documents` 클래스 메서드는 문서 리스트로부터 벡터 저장소를 생성합니다. 

**매개변수**

- `documents` (List[Document]): 벡터 저장소에 추가할 문서 리스트
- `embedding` (Optional[Embeddings]): 임베딩 함수. 기본값은 None
- `ids` (Optional[List[str]]): 문서 ID 리스트. 기본값은 None
- `collection_name` (str): 생성할 컬렉션 이름.
- `persist_directory` (Optional[str]): 컬렉션을 저장할 디렉토리. 기본값은 None
- `client_settings` (Optional[chromadb.config.Settings]): Chroma 클라이언트 설정
- `client` (Optional[chromadb.Client]): Chroma 클라이언트 인스턴스
- `collection_metadata` (Optional[Dict]): 컬렉션 구성 정보. 기본값은 None

**참고**

- `persist_directory`가 지정되면 컬렉션이 해당 디렉토리에 저장됩니다. 지정되지 않으면 데이터는 메모리에 임시로 저장됩니다.
- 이 메서드는 내부적으로 `from_texts` 메서드를 호출하여 벡터 저장소를 생성합니다.
- 문서의 `page_content`는 텍스트로, `metadata`는 메타데이터로 사용됩니다.

**반환값**

- `Chroma`: 생성된 Chroma 벡터 저장소 인스턴스

생성시 `documents` 매개변수로 `Document` 리스트를 전달합니다. embedding 에 활용할 임베딩 모델을 지정하며, `namespace` 의 역할을 하는 `collection_name` 을 지정할 수 있습니다.


In [ ]:
# DB 생성
db_doc = Chroma.from_documents(
    documents=split_doc1, embedding=embeddings, collection_name="my_db2"
)

`persist_directory` 지정시 disk 에 파일 형태로 저장합니다.

In [ ]:
# 저장할 경로 지정
DB_PATH = "./chroma_db2"

# 문서를 디스크에 저장합니다. 저장시 persist_directory에 저장할 경로를 지정합니다.
persist_db = Chroma.from_documents(
    split_doc1, embeddings, persist_directory=DB_PATH, collection_name="my_db2"
)

아래의 코드를 실행하여 `DB_PATH` 에 저장된 데이터를 로드합니다.

In [ ]:
# 디스크에서 문서를 로드합니다.
persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embeddings,
    collection_name="my_db2",
)

불러온 VectorStore 에서 저장된 데이터를 확인합니다.

In [ ]:
# 저장된 데이터 확인
persist_db.get()

{'ids': ['771c0dd0-207d-43fe-b4e2-3b2405544a0b',
  '4de1c04b-ff3c-4c28-89db-0b65f88a6ac3',
  '0853b9f2-068e-4a74-a806-f48e62b7fe7d',
  '871e7ffe-9817-4858-9e6c-874be323bbba',
  '88b7ab46-943c-4eec-821a-4e2a8fc180bd',
  'b7ec9d06-d270-476b-b750-ecfb72ce65e4',
  'c5320686-5b75-41f4-a02a-0a06dfdb0c75',
  '0d028c6c-cc22-4d53-8b1a-44f0ae2f878d',
  '45bbcb56-2edc-4c94-b9a8-60d30448ffb2',
  '0a852462-748d-4f3a-907f-7290495058f4',
  'e1b08bce-2de1-4ca1-8ca2-b46639ed1490'],
 'embeddings': None,
 'documents': ['Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 

만약 `collection_name` 을 다르게 지정하면 저장된 데이터가 없기 때문에 아무런 결과도 얻지 못합니다.

In [ ]:
# 디스크에서 문서를 로드합니다.
persist_db2 = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embeddings,
    collection_name="my_db3",
)

# 저장된 데이터 확인
persist_db2.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### 벡터 저장소 생성 (from_texts)

`from_texts` 클래스 메서드는 텍스트 리스트로부터 벡터 저장소를 생성합니다.

**매개변수**

- `texts` (List[str]): 컬렉션에 추가할 텍스트 리스트
- `embedding` (Optional[Embeddings]): 임베딩 함수. 기본값은 None
- `metadatas` (Optional[List[dict]]): 메타데이터 리스트. 기본값은 None
- `ids` (Optional[List[str]]): 문서 ID 리스트. 기본값은 None
- `collection_name` (str): 생성할 컬렉션 이름. 기본값은 '_LANGCHAIN_DEFAULT_COLLECTION_NAME'
- `persist_directory` (Optional[str]): 컬렉션을 저장할 디렉토리. 기본값은 None
- `client_settings` (Optional[chromadb.config.Settings]): Chroma 클라이언트 설정
- `client` (Optional[chromadb.Client]): Chroma 클라이언트 인스턴스
- `collection_metadata` (Optional[Dict]): 컬렉션 구성 정보. 기본값은 None

**참고**

- `persist_directory`가 지정되면 컬렉션이 해당 디렉토리에 저장됩니다. 지정되지 않으면 데이터는 메모리에 임시로 저장됩니다.
- `ids`가 제공되지 않으면 UUID를 사용하여 자동으로 생성됩니다.

**반환값**

- 생성된 벡터 저장소 인스턴스

In [ ]:
# 문자열 리스트로 생성
db_text = Chroma.from_texts(
    ["안녕하세요. 정말 반갑습니다.", "제 이름은 보경입니다."],
    embedding=embeddings,
    collection_name="my_db4"
)

In [ ]:
# 데이터를 조회합니다.
db_text.get()

{'ids': ['bfdf22c4-8e2d-43c0-b847-38765ab277e5',
  'efa788cb-6c75-4dbe-9895-aaee498ed365',
  'ef1aa2a5-fbd7-4f92-bc97-e2320c70afca',
  'dca75dbe-a65e-485b-ac5a-4abfec15565f'],
 'embeddings': None,
 'documents': ['안녕하세요. 정말 반갑습니다.',
  '제 이름은 보경입니다.',
  '안녕하세요. 정말 반갑습니다.',
  '제 이름은 보경입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [None, None, None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### 유사도 검색

`similarity_search` 메서드는 Chroma 데이터베이스에서 유사도 검색을 수행합니다. 이 메서드는 주어진 쿼리와 가장 유사한 문서들을 반환합니다.

**매개변수**

- `query` (str): 검색할 쿼리 텍스트
- `k` (int, 선택적): 반환할 결과의 수. 기본값은 4입니다.
- `filter` (Dict[str, str], 선택적): 메타데이터로 필터링. 기본값은 None입니다.

**참고**

- `k` 값을 조절하여 원하는 수의 결과를 얻을 수 있습니다.
- `filter` 매개변수를 사용하여 특정 메타데이터 조건에 맞는 문서만 검색할 수 있습니다.
- 이 메서드는 점수 정보 없이 문서만 반환합니다. 점수 정보도 필요한 경우 `similarity_search_with_score` 메서드를 직접 사용하세요.

**반환값**

- `List[Document]`: 쿼리 텍스트와 가장 유사한 문서들의 리스트

In [ ]:
db_doc.similarity_search("TF IDF 에 대하여 알려줘")

[Document(id='fa3e44ce-3829-464d-b033-b1bc336225d5', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='f1ccf401-a562-4277-a1c8-232e608c4f79', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이

`k` 값에 검색 결과의 개수를 지정할 수 있습니다.

In [ ]:
db_doc.similarity_search("TF IDF 에 대하여 알려줘", k=2)

[Document(id='fa3e44ce-3829-464d-b033-b1bc336225d5', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='f1ccf401-a562-4277-a1c8-232e608c4f79', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이

`filter` 에 `metadata` 정보를 활용하여 검색 결과를 필터링 할 수 있습니다.

In [ ]:
# filter 사용
db_doc.similarity_search(
    "TF IDF 에 대하여 알려줘", filter={"source": "./data/vector_store/nlp-keywords.txt"}, k=2
)

[Document(id='fa3e44ce-3829-464d-b033-b1bc336225d5', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='f1ccf401-a562-4277-a1c8-232e608c4f79', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이

다음은 `filter` 에서 다른 `source` 를 사용하여 검색한 결과를 확인합니다.

In [ ]:
# filter 사용
db_doc.similarity_search(
    "TF IDF 에 대하여 알려줘", filter={"source": "./data/vector_store/finance-keywords.txt"}, k=2
)

[]

### 벡터 저장소에 문서 추가

`add_documents` 메서드는 벡터 저장소에 문서를 추가하거나 업데이트합니다.

**매개변수**

- `documents` (List[Document]): 벡터 저장소에 추가할 문서 리스트
- `**kwargs`: 추가 키워드 인자
  - `ids`: 문서 ID 리스트 (제공 시 문서의 ID보다 우선함)

**참고**

- `add_texts` 메서드가 구현되어 있어야 합니다.
- 문서의 `page_content`는 텍스트로, `metadata`는 메타데이터로 사용됩니다.
- 문서에 ID가 있고 `kwargs`에 ID가 제공되지 않으면 문서의 ID가 사용됩니다.
- `kwargs`의 ID와 문서 수가 일치하지 않으면 ValueError가 발생합니다.

**반환값**

- `List[str]`: 추가된 텍스트의 ID 리스트

**예외**

- `NotImplementedError`: `add_texts` 메서드가 구현되지 않은 경우 발생

In [ ]:
from langchain_core.documents import Document

# page_content, metadata, id 지정
db_doc.add_documents(
    [
        Document(
            page_content="안녕하세요! 이번엔 도큐먼트를 새로 추가해 볼께요",
            metadata={"source": "mydata.txt"},
            id="1",
        )
    ]
)

['1']

In [ ]:
# id=1 로 문서 조회
db_doc.get("1")

{'ids': ['1'],
 'embeddings': None,
 'documents': ['안녕하세요! 이번엔 도큐먼트를 새로 추가해 볼께요'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

`add_texts` 메서드는 텍스트를 임베딩하고 벡터 저장소에 추가합니다.

**매개변수**

- `texts` (Iterable[str]): 벡터 저장소에 추가할 텍스트 리스트
- `metadatas` (Optional[List[dict]]): 메타데이터 리스트. 기본값은 None
- `ids` (Optional[List[str]]): 문서 ID 리스트. 기본값은 None

**참고**

- `ids`가 제공되지 않으면 UUID를 사용하여 자동으로 생성됩니다.
- 임베딩 함수가 설정되어 있으면 텍스트를 임베딩합니다.
- 메타데이터가 제공된 경우:
  - 메타데이터가 있는 텍스트와 없는 텍스트를 분리하여 처리합니다.
  - 메타데이터가 없는 텍스트의 경우 빈 딕셔너리로 채웁니다.
- 컬렉션에 upsert 작업을 수행하여 텍스트, 임베딩, 메타데이터를 추가합니다.

**반환값**

- `List[str]`: 추가된 텍스트의 ID 리스트

**예외**

- `ValueError`: 복잡한 메타데이터로 인한 오류 발생 시, 필터링 방법 안내 메시지와 함께 발생

기존의 아이디에 추가하는 경우 `upsert` 가 수행되며, 기존의 문서는 대체됩니다.

In [ ]:
# 신규 데이터를 추가합니다. 이때 기존의 id=1 의 데이터는 덮어쓰게 됩니다.
db_doc.add_texts(
    ["이전에 추가한 Document 를 덮어쓰겠습니다.", "덮어쓴 결과가 어떤가요?"],
    metadatas=[{"source": "mydata.txt"}, {"source": "mydata.txt"}],
    ids=["1", "2"],
)

['1', '2']

In [ ]:
# id=1 조회
db_doc.get(["1"])

{'ids': ['1'],
 'embeddings': None,
 'documents': ['이전에 추가한 Document 를 덮어쓰겠습니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### 벡터 저장소에서 문서 삭제

`delete` 메서드는 벡터 저장소에서 지정된 ID의 문서를 삭제합니다.

**매개변수**

- `ids` (Optional[List[str]]): 삭제할 문서의 ID 리스트. 기본값은 None

**참고**

- 이 메서드는 내부적으로 컬렉션의 `delete` 메서드를 호출합니다.
- `ids`가 None이면 아무 작업도 수행하지 않습니다.

**반환값**

- None

In [ ]:
# id 1 삭제
db_doc.delete(ids=["1"])

In [ ]:
# 문서 조회
db_doc.get(["1", "2"])

{'ids': ['2'],
 'embeddings': None,
 'documents': ['덮어쓴 결과가 어떤가요?'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [ ]:
# where 조건으로 metadata 조회
db_doc.get(where={"source": "mydata.txt"})

{'ids': ['2'],
 'embeddings': None,
 'documents': ['덮어쓴 결과가 어떤가요?'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### 초기화(reset_collection)

`reset_collection` 메서드는 벡터 저장소의 컬렉션을 초기화합니다.


In [ ]:
# 컬렉션 초기화
db_doc.reset_collection()

In [ ]:
# 초기화 후 문서 조회
db_doc.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

# FAISS


Facebook AI Similarity Search (Faiss)는 밀집 벡터의 효율적인 유사도 검색과 클러스터링을 위한 라이브러리입니다.

Faiss는 RAM에 맞지 않을 수도 있는 벡터 집합을 포함하여 모든 크기의 벡터 집합을 검색하는 알고리즘을 포함하고 있습니다.

또한 평가와 매개변수 튜닝을 위한 지원 코드도 포함되어 있습니다.

**참고**
- [LangChain FAISS 문서](https://python.langchain.com/v0.2/docs/integrations/vectorstores/faiss/)
- [FAISS 문서](https://faiss.ai/)

In [ ]:
#실습용 AOAI 환경변수 읽기
import os

AOAI_ENDPOINT=os.getenv("AOAI_ENDPOINT")
AOAI_API_KEY=os.getenv("AOAI_API_KEY")
AOAI_DEPLOY_GPT4O=os.getenv("AOAI_DEPLOY_GPT4O")
AOAI_DEPLOY_GPT4O_MINI=os.getenv("AOAI_DEPLOY_GPT4O_MINI")
AOAI_DEPLOY_EMBED_3_LARGE=os.getenv("AOAI_DEPLOY_EMBED_3_LARGE")
AOAI_DEPLOY_EMBED_3_SMALL=os.getenv("AOAI_DEPLOY_EMBED_3_SMALL")
AOAI_DEPLOY_EMBED_ADA=os.getenv("AOAI_DEPLOY_EMBED_ADA")

샘플 데이터셋을 로드합니다.

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("./data/vector_store/nlp-keywords.txt")
loader2 = TextLoader("./data/vector_store/finance-keywords.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1), len(split_doc2)

(11, 6)

### FAISS 벡터 저장소 생성 (from_documents)

`from_documents` 클래스 메서드는 문서 리스트와 임베딩 함수를 사용하여 FAISS 벡터 저장소를 생성합니다.

**매개변수**

- `documents` (List[Document]): 벡터 저장소에 추가할 문서 리스트
- `embedding` (Embeddings): 사용할 임베딩 함수
- `**kwargs`: 추가 키워드 인자

**동작 방식**

1. 문서 리스트에서 텍스트 내용(`page_content`)과 메타데이터를 추출합니다.
2. 추출한 텍스트와 메타데이터를 사용하여 `from_texts` 메서드를 호출합니다.

**반환값**

- `VectorStore`: 문서와 임베딩으로 초기화된 벡터 저장소 인스턴스

**참고**

- 이 메서드는 `from_texts` 메서드를 내부적으로 호출하여 벡터 저장소를 생성합니다.
- 문서의 `page_content`는 텍스트로, `metadata`는 메타데이터로 사용됩니다.
- 추가적인 설정이 필요한 경우 `kwargs`를 통해 전달할 수 있습니다.

In [ ]:
# OpenAI 임베딩을 생성합니다.
embeddings = AzureOpenAIEmbeddings(model=AOAI_DEPLOY_EMBED_3_LARGE,
    openai_api_version="2024-02-01",
    api_key= AOAI_API_KEY,  
    azure_endpoint=AOAI_ENDPOINT
    )

In [ ]:
#!pip install faiss-cpu


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# DB 생성
split_doc1.extend(split_doc2)
db_doc = FAISS.from_documents(documents=split_doc1, embedding=embeddings)

In [ ]:
# 문서 저장소 ID 확인
db_doc.index_to_docstore_id

{0: '76c6fb3c-b5d2-4dd3-8c15-4b69151472c9',
 1: 'ce76172d-ae7d-45f4-a1aa-ef1a622bfe1f',
 2: '212e54b5-22bd-4b9e-8d6c-1c256fb2dd8f',
 3: 'ac5bac5b-b907-41d5-8c05-4f3e25c03b19',
 4: 'c810ff52-2abb-48a3-90c5-ffb6e61d9b4a',
 5: 'abfd77cf-f48b-47b1-80b9-20ca10c77068',
 6: 'f297e148-3c06-4cab-bc1b-84d4fd428b99',
 7: 'b5b31528-c28e-43f8-9598-0c3637fa4c1d',
 8: '8641f8e4-9888-405b-a3e7-ee40b7950d6f',
 9: 'ad7a237f-e9b7-4acb-97f4-81eb8ae524aa',
 10: 'ab324b30-7bc1-455a-955d-8025fa63b452',
 11: '0fdfb702-0f47-46fc-aead-6ee7d621ebae',
 12: '97ee36e7-dc58-4ba9-a661-ac5661f36619',
 13: '0b22ee18-c720-4aee-bb0b-38b5c666a644',
 14: 'b2b4ba5f-992c-466a-80b9-caa14110adaa',
 15: '8a7e207f-71c9-4f88-8b47-9dfbab576420',
 16: 'b86860c0-2596-4129-8bb2-a572de594720'}

In [ ]:
# 저장된 문서의 ID: Document 확인
db_doc.docstore._dict

{'76c6fb3c-b5d2-4dd3-8c15-4b69151472c9': Document(id='76c6fb3c-b5d2-4dd3-8c15-4b69151472c9', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nTokenizer'),
 'ce76172d-ae7d-45f4-a1aa-ef1a622bfe1f': Document(id='ce76172d-ae7d-45f4-a1aa-ef1a622bfe1f', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.\n예시: "I love programming."이라는 문장을 ["I", "love", "pro

### FAISS 벡터 저장소 생성 (from_texts)

`from_texts` 클래스 메서드는 텍스트 리스트와 임베딩 함수를 사용하여 FAISS 벡터 저장소를 생성합니다.

**매개변수**

- `texts` (List[str]): 벡터 저장소에 추가할 텍스트 리스트
- `embedding` (Embeddings): 사용할 임베딩 함수
- `metadatas` (Optional[List[dict]]): 메타데이터 리스트. 기본값은 None
- `ids` (Optional[List[str]]): 문서 ID 리스트. 기본값은 None
- `**kwargs`: 추가 키워드 인자

**동작 방식**

1. 제공된 임베딩 함수를 사용하여 텍스트를 임베딩합니다.
2. 임베딩된 벡터와 함께 `__from` 메서드를 호출하여 FAISS 인스턴스를 생성합니다.

**반환값**

- `FAISS`: 생성된 FAISS 벡터 저장소 인스턴스

**참고**

- 이 메서드는 사용자 친화적인 인터페이스로, 문서 임베딩, 메모리 내 문서 저장소 생성, FAISS 데이터베이스 초기화를 한 번에 처리합니다.
- 빠르게 시작하기 위한 편리한 방법입니다.

**주의사항**

- 대량의 텍스트를 처리할 때는 메모리 사용량에 주의해야 합니다.
- 메타데이터나 ID를 사용하려면 텍스트 리스트와 동일한 길이의 리스트로 제공해야 합니다.

In [ ]:
# 문자열 리스트로 생성
db_text = FAISS.from_texts(
    ["안녕하세요. 정말 반갑습니다.", "제 이름은 보경입니다."],
    embedding=embeddings,
    metadatas=[{"source": "텍스트문서"}, {"source": "텍스트문서"}],
    ids=["doc1", "doc2"],
)

저장된 결과를 확인합니다. id 값은 지정한 id 값이 잘 들어가 있는지 확인합니다.

In [ ]:
# 저장된 내용
db_text.docstore._dict

{'doc1': Document(id='doc1', metadata={'source': '텍스트문서'}, page_content='안녕하세요. 정말 반갑습니다.'),
 'doc2': Document(id='doc2', metadata={'source': '텍스트문서'}, page_content='제 이름은 보경입니다.')}

### 유사도 검색 (Similarity Search)

`similarity_search` 메서드는 주어진 쿼리와 가장 유사한 문서들을 검색하는 기능을 제공합니다.

**매개변수**

- `query` (str): 유사한 문서를 찾기 위한 검색 쿼리 텍스트
- `k` (int): 반환할 문서 수. 기본값은 4
- `filter` (Optional[Union[Callable, Dict[str, Any]]]): 메타데이터 필터링 함수 또는 딕셔너리. 기본값은 None
- `fetch_k` (int): 필터링 전에 가져올 문서 수. 기본값은 20
- `**kwargs`: 추가 키워드 인자

**반환값**

- `List[Document]`: 쿼리와 가장 유사한 문서 리스트

**동작 방식**

1. `similarity_search_with_score` 메서드를 내부적으로 호출하여 유사도 점수와 함께 문서를 검색합니다.
2. 검색 결과에서 점수를 제외하고 문서만 추출하여 반환합니다.

**주요 특징**

- `filter` 매개변수를 사용하여 메타데이터 기반의 필터링이 가능합니다.
- `fetch_k`를 통해 필터링 전 검색할 문서 수를 조절할 수 있어, 필터링 후 원하는 수의 문서를 확보할 수 있습니다.

**사용 시 고려사항**

- 검색 성능은 사용된 임베딩 모델의 품질에 크게 의존합니다.
- 대규모 데이터셋에서는 `k`와 `fetch_k` 값을 적절히 조정하여 검색 속도와 정확도의 균형을 맞추는 것이 중요합니다.
- 복잡한 필터링이 필요한 경우, `filter` 매개변수에 커스텀 함수를 전달하여 세밀한 제어가 가능합니다.

**최적화 팁**

- 자주 사용되는 쿼리에 대해서는 결과를 캐싱하여 반복적인 검색 속도를 향상시킬 수 있습니다.
- `fetch_k`를 너무 크게 설정하면 검색 속도가 느려질 수 있으므로, 적절한 값을 실험적으로 찾는 것이 좋습니다.

In [ ]:
# 유사도 검색
db_doc.similarity_search("TF IDF 에 대하여 알려줘")

[Document(id='f297e148-3c06-4cab-bc1b-84d4fd428b99', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='abfd77cf-f48b-47b1-80b9-20ca10c77068', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이

`k` 값에 검색 결과의 개수를 지정할 수 있습니다.

In [ ]:
# k 값 지정
db_doc.similarity_search("TF IDF 에 대하여 알려줘", k=2)

[Document(id='f297e148-3c06-4cab-bc1b-84d4fd428b99', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='abfd77cf-f48b-47b1-80b9-20ca10c77068', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이

filter 에 metadata 정보를 활용하여 Filtering 할 수 있습니다.

In [ ]:
# filter 사용
db_doc.similarity_search(
    "TF IDF 에 대하여 알려줘", filter={"source": "./data/vector_store/nlp-keywords.txt"}, k=2
)

[Document(id='f297e148-3c06-4cab-bc1b-84d4fd428b99', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='abfd77cf-f48b-47b1-80b9-20ca10c77068', metadata={'source': './data/vector_store/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이

다음은 `filter` 에서 다른 `source` 를 사용하여 검색한 결과를 확인합니다.

In [ ]:
# filter 사용
db_doc.similarity_search(
    "TF IDF 에 대하여 알려줘", filter={"source": "./data/vector_store/finance-keywords.txt"}, k=2
)

[Document(id='0b22ee18-c720-4aee-bb0b-38b5c666a644', metadata={'source': './data/vector_store/finance-keywords.txt'}, page_content='정의: 주가수익비율(P/E)은 주가를 주당순이익으로 나눈 값으로, 기업의 가치를 평가하는 데 사용되는 지표입니다.\n예시: 아마존의 P/E 비율이 높은 것은 투자자들이 회사의 미래 성장 가능성을 높게 평가하고 있다는 것을 의미합니다.\n연관키워드: 주식 가치평가, 투자 분석, 성장주\n\nQuarterly Earnings Report\n\n정의: 분기별 실적 보고서는 기업이 3개월마다 발표하는 재무 성과와 사업 현황에 대한 보고서입니다.\n예시: 애플의 분기별 실적 발표는 전체 기술 섹터와 S&P 500 지수에 큰 영향을 미칩니다.\n연관키워드: 기업 실적, 투자자 관계, 재무 분석\n\nIndex Fund\n\n정의: 인덱스 펀드는 S&P 500과 같은 특정 지수의 구성과 성과를 그대로 추종하는 투자 상품입니다.\n예시: 바운가드 S&P 500 ETF는 S&P 500 지수를 추종하는 대표적인 인덱스 펀드입니다.\n연관키워드: 패시브 투자, ETF, 포트폴리오 관리\n\nMarket Weight'),
 Document(id='b86860c0-2596-4129-8bb2-a572de594720', metadata={'source': './data/vector_store/finance-keywords.txt'}, page_content='정의: ESG는 기업의 환경, 사회, 지배구조 측면을 고려하는 투자 접근 방식입니다.\n예시: S&P 500 ESG 지수는 우수한 ESG 성과를 보이는 기업들로 구성된 지수입니다.\n연관키워드: 지속가능 투자, 기업의 사회적 책임, 윤리 경영\n\nStock Buyback\n\n정의: 자사주 매입은 기업이 자사의 주식을 시장에서 다시 사들이는 것을 말합니다.\n예시: 애플은 S&P 500 기업 중 가

### 문서(Document)로부터 추가 (add_documents)

`add_documents` 메서드는 벡터 저장소에 문서를 추가하거나 업데이트하는 기능을 제공합니다.

**매개변수**

- `documents` (List[Document]): 벡터 저장소에 추가할 문서 리스트
- `**kwargs`: 추가 키워드 인자

**반환값**

- `List[str]`: 추가된 텍스트의 ID 리스트

**동작 방식**

1. 문서에서 텍스트 내용과 메타데이터를 추출합니다.
2. `add_texts` 메서드를 호출하여 실제 추가 작업을 수행합니다.

**주요 특징**

- 문서 객체를 직접 처리할 수 있어 편리합니다.
- ID 처리 로직이 포함되어 있어 문서의 고유성을 보장합니다.
- `add_texts` 메서드를 기반으로 동작하여 코드 재사용성을 높입니다.

In [ ]:
from langchain_core.documents import Document

# page_content, metadata 지정
db_doc.add_documents(
    [
        Document(
            page_content="안녕하세요! 이번엔 도큐먼트를 새로 추가해 볼께요",
            metadata={"source": "mydata.txt"},
        )
    ],
    ids=["new_doc1"],
)

['new_doc1']

In [ ]:
# 추가된 데이터를 확인
db_doc.similarity_search("안녕하세요", k=1)

[Document(id='new_doc1', metadata={'source': 'mydata.txt'}, page_content='안녕하세요! 이번엔 도큐먼트를 새로 추가해 볼께요')]

### 텍스트로부터 추가 (add_texts)

`add_texts` 메서드는 텍스트를 임베딩하고 벡터 저장소에 추가하는 기능을 제공합니다.

**매개변수**

- `texts` (Iterable[str]): 벡터 저장소에 추가할 텍스트 이터러블
- `metadatas` (Optional[List[dict]]): 텍스트와 연관된 메타데이터 리스트 (선택적)
- `ids` (Optional[List[str]]): 텍스트의 고유 식별자 리스트 (선택적)
- `**kwargs`: 추가 키워드 인자

**반환값**

- `List[str]`: 벡터 저장소에 추가된 텍스트의 ID 리스트

**동작 방식**

1. 입력받은 텍스트 이터러블을 리스트로 변환합니다.
2. `_embed_documents` 메서드를 사용하여 텍스트를 임베딩합니다.
3. `__add` 메서드를 호출하여 임베딩된 텍스트를 벡터 저장소에 추가합니다.

In [ ]:
# 신규 데이터를 추가
db_doc.add_texts(
    ["이번엔 텍스트 데이터를 추가합니다.", "추가한 2번째 텍스트 데이터 입니다."],
    metadatas=[{"source": "mydata.txt"}, {"source": "mydata.txt"}],
    ids=["new_doc2", "new_doc3"],
)

['new_doc2', 'new_doc3']

In [ ]:
# 추가된 데이터를 확인
db_doc.index_to_docstore_id

{0: '76c6fb3c-b5d2-4dd3-8c15-4b69151472c9',
 1: 'ce76172d-ae7d-45f4-a1aa-ef1a622bfe1f',
 2: '212e54b5-22bd-4b9e-8d6c-1c256fb2dd8f',
 3: 'ac5bac5b-b907-41d5-8c05-4f3e25c03b19',
 4: 'c810ff52-2abb-48a3-90c5-ffb6e61d9b4a',
 5: 'abfd77cf-f48b-47b1-80b9-20ca10c77068',
 6: 'f297e148-3c06-4cab-bc1b-84d4fd428b99',
 7: 'b5b31528-c28e-43f8-9598-0c3637fa4c1d',
 8: '8641f8e4-9888-405b-a3e7-ee40b7950d6f',
 9: 'ad7a237f-e9b7-4acb-97f4-81eb8ae524aa',
 10: 'ab324b30-7bc1-455a-955d-8025fa63b452',
 11: '0fdfb702-0f47-46fc-aead-6ee7d621ebae',
 12: '97ee36e7-dc58-4ba9-a661-ac5661f36619',
 13: '0b22ee18-c720-4aee-bb0b-38b5c666a644',
 14: 'b2b4ba5f-992c-466a-80b9-caa14110adaa',
 15: '8a7e207f-71c9-4f88-8b47-9dfbab576420',
 16: 'b86860c0-2596-4129-8bb2-a572de594720',
 17: 'new_doc1',
 18: 'new_doc2',
 19: 'new_doc3'}

### 문서 삭제 (Delete Documents)

`delete` 메서드는 벡터 저장소에서 지정된 ID에 해당하는 문서를 삭제하는 기능을 제공합니다.

**매개변수**

- `ids` (Optional[List[str]]): 삭제할 문서의 ID 리스트
- `**kwargs`: 추가 키워드 인자 (이 메서드에서는 사용되지 않음)

**반환값**

- `Optional[bool]`: 삭제 성공 시 True, 실패 시 False, 구현되지 않은 경우 None

**동작 방식**

1. 입력된 ID의 유효성을 검사합니다.
2. 삭제할 ID에 해당하는 인덱스를 찾습니다.
3. FAISS 인덱스에서 해당 ID를 제거합니다.
4. 문서 저장소에서 해당 ID의 문서를 삭제합니다.
5. 인덱스와 ID 매핑을 업데이트합니다.

**주요 특징**

- ID 기반 삭제로 정확한 문서 관리가 가능합니다.
- FAISS 인덱스와 문서 저장소 양쪽에서 삭제를 수행합니다.
- 삭제 후 인덱스 재정렬을 통해 데이터 일관성을 유지합니다.

**주의사항**

- 삭제 작업은 되돌릴 수 없으므로 신중하게 수행해야 합니다.
- 동시성 제어가 구현되어 있지 않아 다중 스레드 환경에서 주의가 필요합니다.

In [ ]:
# 삭제용 데이터를 추가
ids = db_doc.add_texts(
    ["삭제용 데이터를 추가합니다.", "2번째 삭제용 데이터입니다."],
    metadatas=[{"source": "mydata.txt"}, {"source": "mydata.txt"}],
    ids=["delete_doc1", "delete_doc2"],
)

In [ ]:
# 삭제할 id 를 확인
print(ids)

['delete_doc1', 'delete_doc2']


- `delete` 는 ids 를 입력하여 삭제할 수 있습니다.

In [ ]:
db_doc.index_to_docstore_id

{0: '76c6fb3c-b5d2-4dd3-8c15-4b69151472c9',
 1: 'ce76172d-ae7d-45f4-a1aa-ef1a622bfe1f',
 2: '212e54b5-22bd-4b9e-8d6c-1c256fb2dd8f',
 3: 'ac5bac5b-b907-41d5-8c05-4f3e25c03b19',
 4: 'c810ff52-2abb-48a3-90c5-ffb6e61d9b4a',
 5: 'abfd77cf-f48b-47b1-80b9-20ca10c77068',
 6: 'f297e148-3c06-4cab-bc1b-84d4fd428b99',
 7: 'b5b31528-c28e-43f8-9598-0c3637fa4c1d',
 8: '8641f8e4-9888-405b-a3e7-ee40b7950d6f',
 9: 'ad7a237f-e9b7-4acb-97f4-81eb8ae524aa',
 10: 'ab324b30-7bc1-455a-955d-8025fa63b452',
 11: '0fdfb702-0f47-46fc-aead-6ee7d621ebae',
 12: '97ee36e7-dc58-4ba9-a661-ac5661f36619',
 13: '0b22ee18-c720-4aee-bb0b-38b5c666a644',
 14: 'b2b4ba5f-992c-466a-80b9-caa14110adaa',
 15: '8a7e207f-71c9-4f88-8b47-9dfbab576420',
 16: 'b86860c0-2596-4129-8bb2-a572de594720',
 17: 'new_doc1',
 18: 'new_doc2',
 19: 'new_doc3',
 20: 'delete_doc1',
 21: 'delete_doc2'}

In [ ]:
# id 로 삭제
db_doc.delete(ids)

True

In [ ]:
# 삭제된 결과를 출력
db_doc.index_to_docstore_id

{0: '76c6fb3c-b5d2-4dd3-8c15-4b69151472c9',
 1: 'ce76172d-ae7d-45f4-a1aa-ef1a622bfe1f',
 2: '212e54b5-22bd-4b9e-8d6c-1c256fb2dd8f',
 3: 'ac5bac5b-b907-41d5-8c05-4f3e25c03b19',
 4: 'c810ff52-2abb-48a3-90c5-ffb6e61d9b4a',
 5: 'abfd77cf-f48b-47b1-80b9-20ca10c77068',
 6: 'f297e148-3c06-4cab-bc1b-84d4fd428b99',
 7: 'b5b31528-c28e-43f8-9598-0c3637fa4c1d',
 8: '8641f8e4-9888-405b-a3e7-ee40b7950d6f',
 9: 'ad7a237f-e9b7-4acb-97f4-81eb8ae524aa',
 10: 'ab324b30-7bc1-455a-955d-8025fa63b452',
 11: '0fdfb702-0f47-46fc-aead-6ee7d621ebae',
 12: '97ee36e7-dc58-4ba9-a661-ac5661f36619',
 13: '0b22ee18-c720-4aee-bb0b-38b5c666a644',
 14: 'b2b4ba5f-992c-466a-80b9-caa14110adaa',
 15: '8a7e207f-71c9-4f88-8b47-9dfbab576420',
 16: 'b86860c0-2596-4129-8bb2-a572de594720',
 17: 'new_doc1',
 18: 'new_doc2',
 19: 'new_doc3'}

## 저장 및 로드

### 로컬 저장 (Save Local)

`save_local` 메서드는 FAISS 인덱스, 문서 저장소, 그리고 인덱스-문서 ID 매핑을 로컬 디스크에 저장하는 기능을 제공합니다.

**매개변수**

- `folder_path` (str): 저장할 폴더 경로
- `index_name` (str): 저장할 인덱스 파일 이름 (기본값: "index")

**동작 방식**

1. 지정된 폴더 경로를 생성합니다 (이미 존재하는 경우 무시).
2. FAISS 인덱스를 별도의 파일로 저장합니다.
3. 문서 저장소와 인덱스-문서 ID 매핑을 pickle 형식으로 저장합니다.

**사용 시 고려사항**

- 저장 경로에 대한 쓰기 권한이 필요합니다.
- 대용량 데이터의 경우 저장 공간과 시간이 상당히 소요될 수 있습니다.
- pickle 사용으로 인한 보안 위험을 고려해야 합니다.

In [ ]:
# 로컬 Disk 에 저장
db_doc.save_local(folder_path="./faiss_db", index_name="faiss_index")

### 로컬에서 불러오기 (Load Local)

`load_local` 클래스 메서드는 로컬 디스크에 저장된 FAISS 인덱스, 문서 저장소, 그리고 인덱스-문서 ID 매핑을 불러오는 기능을 제공합니다.

**매개변수**

- `folder_path` (str): 불러올 파일들이 저장된 폴더 경로
- `embeddings` (Embeddings): 쿼리 생성에 사용할 임베딩 객체
- `index_name` (str): 불러올 인덱스 파일 이름 (기본값: "index")
- `allow_dangerous_deserialization` (bool): pickle 파일 역직렬화 허용 여부 (기본값: False)

**반환값**

- `FAISS`: 로드된 FAISS 객체

**동작 방식**

1. 역직렬화의 위험성을 확인하고 사용자의 명시적 허가를 요구합니다.
2. FAISS 인덱스를 별도로 불러옵니다.
3. pickle을 사용하여 문서 저장소와 인덱스-문서 ID 매핑을 불러옵니다.
4. 불러온 데이터로 FAISS 객체를 생성하여 반환합니다.

In [ ]:
# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path="./faiss_db",
    index_name="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)

In [ ]:
# 로드된 데이터를 확인
loaded_db.index_to_docstore_id

{0: '76c6fb3c-b5d2-4dd3-8c15-4b69151472c9',
 1: 'ce76172d-ae7d-45f4-a1aa-ef1a622bfe1f',
 2: '212e54b5-22bd-4b9e-8d6c-1c256fb2dd8f',
 3: 'ac5bac5b-b907-41d5-8c05-4f3e25c03b19',
 4: 'c810ff52-2abb-48a3-90c5-ffb6e61d9b4a',
 5: 'abfd77cf-f48b-47b1-80b9-20ca10c77068',
 6: 'f297e148-3c06-4cab-bc1b-84d4fd428b99',
 7: 'b5b31528-c28e-43f8-9598-0c3637fa4c1d',
 8: '8641f8e4-9888-405b-a3e7-ee40b7950d6f',
 9: 'ad7a237f-e9b7-4acb-97f4-81eb8ae524aa',
 10: 'ab324b30-7bc1-455a-955d-8025fa63b452',
 11: '0fdfb702-0f47-46fc-aead-6ee7d621ebae',
 12: '97ee36e7-dc58-4ba9-a661-ac5661f36619',
 13: '0b22ee18-c720-4aee-bb0b-38b5c666a644',
 14: 'b2b4ba5f-992c-466a-80b9-caa14110adaa',
 15: '8a7e207f-71c9-4f88-8b47-9dfbab576420',
 16: 'b86860c0-2596-4129-8bb2-a572de594720',
 17: 'new_doc1',
 18: 'new_doc2',
 19: 'new_doc3'}